# Depth of the book basis point spread

Component generates a signal from depth of the book spread and provides a measurement of current depth of book spread. Components consuming this event can use this signal to estimate market mid, market spread and other factors.


This component should produce three values

- DOB spread in basis points, instantaneous dob passed through EMA filter
- Above value is normalized using historical baseline. Resulting ratio is adjusted by two configurable parameters, referred to as the ratio signal
- The value from above is floored by the desired minimum hourly normalized dob spread ratio, referred to as the 'bounded ratio signal'



## Component config

- Instrument Id
- Venue Id
- Trades per day for the venue
- dob_spreadBasisPointBaseline
- hourlyMinimumSpreadBasisPointRatio
- dob_spreadBasisPointRatioDamp
- spreadRatioDampCcyAdjust


## Inputs

* Order Book updates
* Trade Updates
* Market data status


## EMA to smooth out dob signal


## Calc Triggers

* Order book updates
* Trades - only as a signal
* Mkt data status


## calc

Calculates the slope of the vwap price ladder upto the max volume of interest. 

### Formula

$ dobSpread = \frac{(vwapAsk - vwapBid)}{\frac{mid}{vwapVolumne}} = \frac{(vwapAsk - vwapBid)} {\frac{[(vwapAsk + vwapBid)/2]}{vwapVolumne}}$

Max voluma used in the vwap calculation is bounded by the minimum available total size on either bid or ask side of the book and the configurable maximum volume of the interest of the currency pair

Pass above dobSpread to EMA filter. Output of the filter is output of this component. 

Signal is updated and published each time a trade notification is received from a configured venue. When the trade notification is received the last order book received by the processor for the venue is used to calculate the spread.




$spreadRatio = \frac{dob\_spread\_basis\_pt}{dob\_spread\_basis\_pt\_baseline}$

$1.0 < spreadRatio < 10 * hourly\_min\_spread\_bp$

$spread\_ratio\_impact = spreadRatio^{ (dob\_spread\_bp\_ratio\_damp + spreadRatio\_damp\_ccy\_adjust)}$

spread_ratio_impact > hourly_min_spread_bp


## Steps

* totalBidAskVol - Find min of bid / ask volumes
* maxVolume - Find min of configured volume of interest and result from above
* vwapBid price at maxVolume
* vwapAsk price at maxVolume
* vwapSpread - 2 * (vwapAsk - vwapBid) / (bestBid + bestAsk)
* dobSprdBasisPt - 1e10 * vwapSpread / maxVolume
* normalize dobSprdBasisPt
* Publish - spreadSignal, ratioSignal, boundedRatioSignal


## Output

* spreadSignal - EMA smoothed spread
* ratioSignal
* boundedRatioSignal

## Uses

* Spread Factors Calculation
* In Market readiness, dob availability check 
* Quote Mid Calculator to check tobSprdBasisPt > multiplierFromConfig * dobSprdBasisPt  